# Preprocesamiento de datos

In [1]:
import numpy as np
import pandas as pd
from tabulate import tabulate
import json
import os

## Lectura y preparación inicial de los datos

### Cargar el dataset

In [2]:
file_path = "../data/raw/data.csv"
df = pd.read_csv(file_path, header=None)

### Preparar los datos

In [3]:
# Crear nombres cortos para las características
feature_names = [f'feat{str(i+1).zfill(2)}' for i in range(30)]

# Eliminar columna ID y asignar nombres a las columnas
df = df.drop([0], axis=1)  # El ID no aporta información
df.columns = ['diagnosis'] + feature_names

# Mapear diagnóstico de M/B a 0/1
df['diagnosis'] = df['diagnosis'].map({'M': 0, 'B': 1})

## Dividir en características (X) y target (y)

In [4]:
X = df[feature_names]
y = df['diagnosis']

## Dividir en conjuntos de entrenamiento y test
- Proponemos estos porcentajes para la división `train - test`: **80% - 20%**
- Establecemos los porcentajes como parámetros.
- Esta división se realiza habitualmente con esta función de la librería 'sklearn'
```python
from sklearn.model_selection import train_test_split
```
Esta implementación replica de forma "manual" la funcionalidad de `train_test_split` de `sklearn` con las siguientes características:
- Permite definir el tamaño del conjunto de test (test_size)
- Implementa la opción de estratificación (stratify) para mantener la proporción de clases
- Permite fijar una semilla aleatoria (random_state) para asegurar reproducibilidad
- Funciona tanto con arrays de NumPy como con listas Python

La implementación estratificada funciona así:
- Agrupa los índices por clase
- Para cada clase, mezcla aleatoriamente sus índices
- Divide los índices de cada clase según la proporción deseada
- Combina los índices para formar los conjuntos de entrenamiento y test

La implementación sin estratificación simplemente mezcla todos los índices y los divide según la proporción deseada.

In [5]:
import numpy as np
from collections import defaultdict

def manual_train_test_split(X, y, test_size=0.2, random_state=None, stratify=None):
    """
    Implementación manual de train_test_split
    
    Parámetros:
    X : Características
    y : Etiquetas o variables objetivo
    test_size : Proporción de datos para el conjunto de prueba (por defecto 0.2)
    random_state : Semilla para la generación de números aleatorios (por defecto None)
    stratify : Array para estratificación (por defecto None)
    
    Retorna:
    X_train, X_test, y_train, y_test
    """
    # Establecer la semilla aleatoria para reproducibilidad
    if random_state is not None:
        np.random.seed(random_state)
    
    # Asegurarse de que X e y tienen la misma longitud
    assert len(X) == len(y), "X e y deben tener la misma longitud"
    
    # Índices de los datos
    indices = np.arange(len(X))
    
    if stratify is not None:
        # Implementación con estratificación
        # Agrupar índices por clase
        indices_por_clase = defaultdict(list)
        for i, clase in enumerate(stratify):
            indices_por_clase[clase].append(i)
        
        # Inicializar los índices de train y test
        train_indices = []
        test_indices = []
        
        # Para cada clase, dividir los índices según la proporción test_size
        for clase, indices_clase in indices_por_clase.items():
            # Mezclar los índices de esta clase
            indices_clase = np.array(indices_clase)
            np.random.shuffle(indices_clase)
            
            # Calcular cuántos elementos van al conjunto de test
            n_test = int(len(indices_clase) * test_size)
            
            # Dividir los índices
            test_indices.extend(indices_clase[:n_test])
            train_indices.extend(indices_clase[n_test:])
    else:
        # Implementación sin estratificación
        # Mezclar todos los índices
        np.random.shuffle(indices)
        
        # Calcular cuántos elementos van al conjunto de test
        n_test = int(len(indices) * test_size)
        
        # Dividir los índices
        test_indices = indices[:n_test]
        train_indices = indices[n_test:]
    
    # Ordenar los índices (no es estrictamente necesario, pero ayuda a la reproducibilidad)
    train_indices = sorted(train_indices)
    test_indices = sorted(test_indices)
    
    # Seleccionar los datos según los índices
    import pandas as pd
    
    if isinstance(X, np.ndarray):
        X_train = X[train_indices]
        X_test = X[test_indices]
    elif isinstance(X, pd.DataFrame):
        X_train = X.iloc[train_indices]
        X_test = X.iloc[test_indices]
    else:
        X_train = [X[i] for i in train_indices]
        X_test = [X[i] for i in test_indices]
    
    if isinstance(y, np.ndarray):
        y_train = y[train_indices]
        y_test = y[test_indices]
    elif isinstance(y, pd.Series):
        y_train = y.iloc[train_indices]
        y_test = y.iloc[test_indices]
    else:
        y_train = [y[i] for i in train_indices]
        y_test = [y[i] for i in test_indices]
    
    return X_train, X_test, y_train, y_test

## Aplicando el split estratificado

In [6]:
# Establecemos el parámetro de porcentaje
train_size = 0.8
test_size = 1 - train_size

X_train, X_test, y_train, y_test = manual_train_test_split(
 X, y, 
 test_size=test_size, 
 random_state=42,
 stratify=y    # Mantener la proporción de clases en ambos conjuntos
)

## Normalización Z-score usando SOLO los datos de entrenamiento

In [7]:
# Calcular media y desviación estándar del conjunto de entrenamiento
mean_train = X_train.mean()
std_train = X_train.std()

# Normalizar conjuntos de entrenamiento y test
X_train_normalized = (X_train - mean_train) / std_train
X_test_normalized = (X_test - mean_train) / std_train

# 6. Guardar los parámetros de normalización
normalization_params = {
    'mean': mean_train,
    'std': std_train
}

## Crear directorios si no existen

In [8]:
processed_dir = "../data/processed"
output_dir = "../output"
os.makedirs(processed_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

## Guardar los conjuntos normalizados en data/processed

In [9]:
# Conjuntos de entrenamiento
train_df = pd.DataFrame(X_train_normalized, columns=feature_names)
train_df.insert(0, 'diagnosis', y_train.values)
train_df.to_csv(f"{processed_dir}/train_normalized.csv", index=False)

# Conjuntos de test
test_df = pd.DataFrame(X_test_normalized, columns=feature_names)
test_df.insert(0, 'diagnosis', y_test.values)
test_df.to_csv(f"{processed_dir}/test_normalized.csv", index=False)

# Guardar los parámetros de normalización en output
# Convertir los parámetros a un diccionario para JSON

# Reorganizar parámetros por característica
params_by_feature = {}

# Para cada característica, guardar su media y desviación en un mismo objeto
for feature in feature_names:
    params_by_feature[feature] = {
        'mean': float(mean_train[feature]),  # Convertir a float para asegurar serialización JSON
        'std': float(std_train[feature])
    }

# Guardar los parámetros en formato JSON
json_path = f"{output_dir}/normalization_params.json"
with open(json_path, 'w') as f:
    json.dump(params_by_feature, f, indent=4)

print(f"\nParámetros de normalización (agrupados por característica) guardados en: {json_path}")


Parámetros de normalización (agrupados por característica) guardados en: ../output/normalization_params.json


## Imprimir información sobre los conjuntos de datos

In [10]:
print("\nInformación sobre la división de datos:")
print(f"Tamaño total del dataset: {len(df)}")
print(f"Tamaño del conjunto de entrenamiento: {len(X_train)}")
print(f"Tamaño del conjunto de test: {len(X_test)}")

# Distribución de clases
print("\nDistribución de clases:")
print("Conjunto de entrenamiento:")
print(y_train.value_counts(normalize=True).round(3))
print("\nConjunto de test:")
print(y_test.value_counts(normalize=True).round(3))

# Mostrar primeras filas del conjunto de entrenamiento normalizado
print("\nPrimeras filas del conjunto de entrenamiento normalizado:")
print(tabulate(train_df.head(), headers='keys', tablefmt='rst', showindex=True))


Información sobre la división de datos:
Tamaño total del dataset: 569
Tamaño del conjunto de entrenamiento: 456
Tamaño del conjunto de test: 113

Distribución de clases:
Conjunto de entrenamiento:
diagnosis
1    0.627
0    0.373
Name: proportion, dtype: float64

Conjunto de test:
diagnosis
1    0.628
0    0.372
Name: proportion, dtype: float64

Primeras filas del conjunto de entrenamiento normalizado:
====  ===========  =========  =========  ========  =========  =========  =========  ==========  ========  ==========  =========  ========  =========  ========  =========  =========  ==========  =========  ========  =========  =========  =========  ==========  =========  =========  =========  =========  =========  ========  =========  =========
  ..    diagnosis     feat01     feat02    feat03     feat04     feat05     feat06      feat07    feat08      feat09     feat10    feat11     feat12    feat13     feat14     feat15      feat16     feat17    feat18     feat19     feat20     feat21  